### Install package

In [1]:
!pip install pytorch-adapt

     |████████████████████████████████| 137 kB 8.9 MB/s 
     |████████████████████████████████| 107 kB 44.1 MB/s 
     |████████████████████████████████| 397 kB 43.8 MB/s 


### Import packages

In [6]:
import torch
from tqdm import tqdm

from pytorch_adapt.containers import Models, Optimizers
from pytorch_adapt.datasets import DataloaderCreator, get_mnist_mnistm
from pytorch_adapt.hooks import DANNHook
from pytorch_adapt.models import Discriminator, mnistC, mnistG
from pytorch_adapt.utils.common_functions import batch_to_device
from pytorch_adapt.validators import IMValidator

### Create datasets and dataloaders

In [7]:
datasets = get_mnist_mnistm(["mnist"], ["mnistm"], folder=".", download=True)
dc = DataloaderCreator(batch_size=32, num_workers=2)
dataloaders = dc(**datasets)

### Create models, optimizers, hook, and validator

In [8]:
device = torch.device("cuda")

G = mnistG(pretrained=True).to(device)
C = mnistC(pretrained=True).to(device)
D = Discriminator(in_size=1200, h=256).to(device)
models = Models({"G": G, "C": C, "D": D})
optimizers = Optimizers((torch.optim.Adam, {"lr": 0.0001}))
optimizers.create_with(models)
optimizers = list(optimizers.values())

hook = DANNHook(optimizers)
validator = IMValidator()

### Train and evaluate

In [9]:
for epoch in range(2):

    # train loop
    models.train()
    for data in tqdm(dataloaders["train"]):
        data = batch_to_device(data, device)
        _, loss = hook({**models, **data})

    # eval loop
    models.eval()
    logits = []
    with torch.no_grad():
        for data in tqdm(dataloaders["target_train"]):
            data = batch_to_device(data, device)
            logits.append(C(G(data["target_imgs"])))
        logits = torch.cat(logits, dim=0)

    # validation score
    score = validator(target_train={"logits": logits})
    print(f"\nEpoch {epoch} score = {score}\n")

100%|██████████| 1844/1844 [00:21<00:00, 86.78it/s]



Epoch 0 score = 1.1073559522628784



100%|██████████| 1844/1844 [00:21<00:00, 87.57it/s]


Epoch 1 score = 1.19959557056427

